# QCoDeS example with Galil DMC4133 Controller

Purpose of this notebook is to demonstrate how Galil DMC4133 Controller driver along with Arm class can be used for running measurements while controlling the arm head simulteneously. Before begining,

1. Make sure that you have `gclib` package installed in your environment. If not, then follow the instructions [here](https://www.galil.com/sw/pub/all/doc/gclib/html/python.html) for installation.
2. Make sure that the controller is connected to your PC through an Ethernet cable and the configuration is set as according to these instructions on Windows operating system.
    
        a. Go to Control Panel -> Network and Internet -> Network Connections and select the appropriate network adapter.
        b. Next go to the Properties of that adapter, and then Properties for Internet Protocol Version 4 (TCP/IPv4).
        c. Select "Use the following IP address" and add an IP address and Subnet. (If the Galil has an IP address of 10.10.10.100 burned in, you would need a PC IP address of something like 10.10.10.1 with a subnet of 255.255.255.0.)

Once, the connection to the DMC4133 Controller is established, we can begin with necessary imports and the calibration process. So, let us begin!

# Imports

In [1]:
from qcodes.instrument_drivers.Galil.dmc_41x3 import DMC4133Controller, Arm

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\CryoProbeusr\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\CryoProbeusr\.qcodes\logs\210906-6460-qcodes.log


In [2]:
controller = DMC4133Controller(name='controller', address='192.168.8.74')

Connected to: Galil Motion Control, Inc. DMC4133 (serial:16639, firmware:1.3f) in 0.03s


In [3]:
arm = Arm(controller)

Now that we have imported an initialized the controller and the arm. We need to calibrate the arm.

# Calibration

Check the state of the motors.

In [62]:
print(controller.absolute_position())
print(controller.motor_a.on_off_status())
print(controller.motor_b.on_off_status())
print(controller.motor_c.on_off_status())

{'A': -90000, 'B': -10000, 'C': -16000}
on
on
on


In [5]:
controller.motors_off()

In [6]:
arm.set_arm_kinematics()  # sets default values of arm speed to be 100 micro meters per second,
                          # acceleration and deceleration to be 2048 micro meters per second square

Manually move the motors and take the needle head to the position where you want to set the origin. This will be the left bottom corner of the chip and run following commands.

In [7]:
controller.define_position_as_origin()

In [54]:
arm.set_left_bottom_position()

From now on all the motor movements will be controlled by the driver commands.

Next step is to set reverse limits for for all three motors. First take the motor C to the extreme reverse position which you want to set as the reverse limit with following command.

In [29]:
arm.move_motor_c_by(distance=-1000)    # distance is in micro meters

Now that you are at the position which you want to set as the reverse limit for motor C, run the following command.

In [30]:
arm.set_motor_c_reverse_limit()

Repeat the same process for motor A.

In [24]:
arm.move_motor_a_by(distance=-200)

In [21]:
arm.set_motor_a_reverse_limit()

Now, for motor B.

In [14]:
arm.move_motor_b_by(distance=-2000)

In [15]:
arm.set_motor_b_reverse_limit()

You have set the reverse limits for all three motors. Next we will define the chip plane. We have already set the chip left bottom corner as the origin of the system. Now, we will set the left top corner first and then right top corner.

Move individual motors with following commands.

In [58]:
arm.move_motor_a_by(distance=-1000)

In [59]:
arm.move_motor_b_by(distance=3000)

In [57]:
arm.move_motor_c_by(distance=-300)

When you are satisfied that the motor is at the left top position of the chip. Run the following command.

In [46]:
arm.set_left_top_position()

Again, move invidual motors with the above mentioned commands and when you are satisfied that the arm needle is at the right top position of the chip, run the following command.

In [60]:
arm.set_right_top_position()

You have not set the boundaries for the motion of the motor. Though the calibration process is not complete yet. You need to set the chip details.

## Set chip details

In [61]:
arm.rows = 2
arm.pads = 3
arm.inter_row_dis = arm.norm_b            # since there are only 2 rows
arm.inter_pad_dis = arm.norm_c / 2        # since there are 3 pads per row

Calibration is complete! Remember you are at the right top position of the chip crrently. Move the needle head to left bottom position and before that set pick up diatance.

In [64]:
arm.set_pick_up_distance()

In [65]:
arm.move_towards_left_bottom_position()

# Integration with measurement process

In [66]:
import qcodes as qc
from qcodes import (
    Measurement,
    initialise_or_create_database_at,
    load_or_create_experiment,
)
from qcodes.tests.instrument_mocks import DummyInstrument, DummyInstrumentWithMeasurement

In [67]:
station = qc.Station()

In [68]:
# A dummy instrument dac with two parameters ch1 and ch2
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])

# A dummy instrument that generates some real looking output depending
# on the values set on the setter_instr, in this case the dac
dmm = DummyInstrumentWithMeasurement('dmm', setter_instr=dac)

In [69]:
station.add_component(dac)
station.add_component(dmm)

'dmm'

In [70]:
initialise_or_create_database_at("~/experiments.db")

Upgrading database; v8 -> v9: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


In [71]:
exp = load_or_create_experiment(experiment_name='galil_controller_testing',
                                sample_name="no sample1")

Our arm is set up and the measurement is set up. Run the following block for measurement as we are at the 1st row.

In [72]:
meas = Measurement(exp=exp, station=station, name='xyz_measurement')
meas.register_parameter(dac.ch1)  # register the first independent parameter
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))  # now register the dependent oone

meas.write_period = 2

with meas.run() as datasaver:
    for set_v in np.linspace(0, 25, 10):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v),
                             (dmm.v1, get_v))

    dataset = datasaver.dataset  # convenient to have for plotting

Starting experimental run with id: 1. 


Now you have option to move to the next row or pad with following commands.

In [78]:
arm.move_to_row(1)

In [74]:
arm.move_to_pad(3)

Once this motion is complete, you can use individual motors commands for minor adjustments. 

In [75]:
arm.move_motor_a_by(distance=15)

In [76]:
arm.move_motor_b_by(distance=50)

In [77]:
arm.move_motor_c_by(distance=-5)

Repeat the measurement code block for the new row or pad.

When you are done with the all the measurements, use following command to close the controller.

In [79]:
arm.controller.close()